In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade -q gspread

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")

In [ ]:
"""
Loop through rows of spreadsheet
Download images
"""
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for row_counter in range(2,254):
    file_num = worksheet_pages.get_values("C"+str(row_counter))[0][0]

In [5]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 8.8 MB/s 
  Created wheel for openai: filename=openai-0.24.0-py3-none-any.whl size=55926 sha256=e50739a330b9bfd6630a87e54cfef328c46c98467c076a74eeefbe40a35ef625
  Stored in directory: /root/.cache/pip/wheels/94/2b/ee/7649ac33c142e3fde2081bd8769337b5e75710fd4b885cd2c6
Successfully built openai


In [6]:
settings_path = "/content/drive/MyDrive/william_white/"
    
import os
import json
settings_file = settings_path+'openai.json'
with open(settings_file, "r") as read_file:
    auth_data = json.load(read_file)

os.environ['OPENAI_API_KEY']=auth_data["OPENAI_API_KEY"]

In [49]:
# This was written by Dr. Shawn Graham - I am adapting this
# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!
# https://medium.com/geekculture/relationship-extraction-with-gpt-3-bb019dcf41e5
# https://github.com/dgg32/gpt-3-extract

# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!

# the wait so that we don't screw up api limits is from the gpt-3 examples repo from openai


import os
import sys
import openai

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


#training_file = '/content/drive/MyDrive/fieldbooks_antioch/shorter-training.txt'

#input_file = '/content/drive/MyDrive/fieldbooks_antioch/sample-test.txt'
training_file = '/content/drive/MyDrive/fieldbooks_antioch/ocr-translation.txt'
input_file = '/content/drive/MyDrive/fieldbooks_antioch/r21.txt'

openai.api_key = os.getenv("OPENAI_API_KEY")

training = "\n".join(open(training_file, 'r').readlines()) + "\n"
text = "\n".join(open(input_file, 'r').readlines()) + "\n"

def extract_relation (text):
  my_prompt = training + text

#  response = openai.Completion.create(
  response = completion_with_backoff(
    engine="text-davinci-002",
    prompt=my_prompt,
    temperature=0.0,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["END"]
  )

  return response.choices[0].text.lstrip()

In [ ]:
with open(input_file, 'r') as file_in:
    for line in file_in:
        if len(line.strip()) > 0:
          res = extract_relation(line.strip() + "\n")

          print(res)

CORRECTION:

C'est un terrain situé à l'Est de la Route ; il est environ l'un des derrières massifs ; planté de quelques figuiers - datant de plusieurs années - il off presseurs ; in ya un remède mais.
Plusieurs tombes voisines, d'environ 5m de profondeur, ont servi à enterrer ; on a extrait des press-ailles, avec des fragments qui gardent une belle apparence. Les travaux ne sont pas encore terminés. Juste de puits voisins à la maison voisine.
3 Juin

TRANSLATION:

This is a piece of land located to the East of the Road; it is one of the back massifs; planted with a few fig trees - dating back several years - it has presses; in there is a remedy but.
Several nearby tombs, about 5m deep, were used for burial; press-ailles were extracted, with fragments that retain a beautiful appearance. The work is not yet finished. Just a few nearby wells to the neighboring house.
June 3
CORRECTION:

C'est un terrain situé à l'Est de la Route ; il est environ l'un des derrières massifs ; planté de que

In [43]:
response = openai.Edit.create(
  model="text-davinci-edit-001",
  input="""3 Juin
# 15h. Toynus pes trace de mouse de travail l'avance
palièrement. La jour Est abe tranchée svangereure .
I y avait une poche de fiers ; les précautions ont pris
pom éviter is éboulements . Le travail ost of anise avec
parcours coventaire par deux camps
In trouve pasmi b débits un fragment de chapation ()
16
5
1
""",
  instruction="Fix OCR errors.",
  temperature=0.7,
  top_p=1
)
#print(response)
print(response.choices[0].text.lstrip())

3 Juin
# 15h. Toynus pes trace de mouse de travail l'avance palièrement. La jour Est abe tranchée svangereure.
I y avait une poche de fiers ; les précautions ont pris pom éviter is éboulements. Le travail ost of anise avec parcours coventaire par deux camps
In trouve pasmi b débits un fragment de chapation () (16/5/1)



In [48]:
response = openai.Edit.create(
  model="text-davinci-edit-001",
  input=""""
hours numming ExW. House wall ar 0.20 helaso surface ) en oito
ovulation wall of hojeps drove wa
· mostly destroyed, alepch of foundation wall. here are. 1.40 de.
Inscription un house et NW corner.
TH Z TIT
KCKOY
Jreal French Hippoknow -A - at 4ch on east side - L3 ml
Irgend sur limite ce: 6. m ready what face + Sim. on Stuck
20 april
N. S.
21 april
Come - bate Roman mostly a late arabic
Trench summing E-W or west side of Supportcome - late a which
wall at wetallend of thand QQ25 m.
Black loom - lenge straks - we pott eng lule buck , file
22 april
Sonic Rap- dia. 0.325 homestore
0~ - 0,35
mutilated
"92
"
""",
  instruction="Fix OCR errors.",
  temperature=0.4,
  top_p=1
)
#print(response)
print(response.choices[0].text.lstrip())

"
hours numming ExW. House wall ar 0.20 helaso surface ) en oito ovulation wall of hojeps drove wa · mostly destroyed, alepch of foundation wall. here are. 1.40 de.
Inscription un house et NW corner. TH Z TIT KCKOY Jreal French Hippoknow -A - at 4ch on east side - L3 ml Irgend sur limite ce: 6. m ready what face + Sim. on Stuck 20 april N. S. 21 april Come - bate Roman mostly a late arabic Trench summing E-W or west side of Supportcome - late a which wall at wetallend of thand QQ25 m. Black loom - lenge straks - we pott eng lule buck , file 22 april Sonic Rap- dia. 0.325 homestore 0~ - 0,35 mutilated "92 "



In [ ]:
            response = openai.Completion.create(
            model="text-davinci-002",
            prompt=generate_prompt(animal),
            temperature=0.6,
        )
        return redirect(url_for("index", result=response.choices[0].text))